## Amazon Review - Zero Shot - Error EDA

This notebook is for the error analysis and eda.

### Load Packages

In [1]:
import pandas as pd
import random
import seaborn as sns
import numpy as np

### Load in Data

In [2]:
file_name = '../../test_predictions.pkl'
test_results = pd.read_pickle(file_name)

file_name = '../../imdb_test_predictions.pkl'
imdb_results = pd.read_pickle(file_name)

file_name = '../../twitter_sent140_test_predictions.pkl'
twitter_results = pd.read_pickle(file_name)

file_name = '../../yelp_test_predictions.pkl'
yelp_results = pd.read_pickle(file_name)

file_name = '../../yelp_zhang_test_predictions.pkl'
yelp_zhang_results = pd.read_pickle(file_name)

file_name = '../../amazon_zhang_test_predictions.pkl'
amazon_zhang_results = pd.read_pickle(file_name)

file_name = '../../sst_test_predictions.pkl'
sst_results = pd.read_pickle(file_name)

In [3]:
def appendPreds(data):
    cols = ['USE_Soft', 'CNN_TE', 'CNN_TE_USE', 'CNN_Glove', 'CNN_Glove_USE',
        'CNN_w2V', 'CNN_w2V_USE', 'Individual_Sent', 'Average_Sent']
    preds = ['USE_Soft_pred', 'CNN_TE_pred', 'CNN_TE_USE_pred', 'CNN_Glove_pred', 'CNN_Glove_USE_pred',
        'CNN_w2V_pred', 'CNN_w2V_USE_pred', 'Individual_Sent_pred', 'Average_Sent_pred']
    
    for col, pred in zip(cols,preds):
        data[pred] = np.where(data[col]>=0.5,1,0)
        
    return data

test_results = appendPreds(test_results)

imdb_results = appendPreds(imdb_results)

twitter_results = appendPreds(twitter_results)

yelp_results = appendPreds(yelp_results)

yelp_zhang_results = appendPreds(yelp_zhang_results)

amazon_zhang_results = appendPreds(amazon_zhang_results)

sst_results = appendPreds(sst_results)

In [4]:
def checkDifs(data, cutoff = 2, Indv = True, Avg = True):
    
    n = data.shape[0]
    newDF = data
    preds = ['USE_Soft_pred', 'CNN_TE_pred', 'CNN_TE_USE_pred', 'CNN_Glove_pred', 'CNN_Glove_USE_pred',
        'CNN_w2V_pred', 'CNN_w2V_USE_pred']
    
    indiv = ['Individual_Sent_pred']
    avg = ['Average_Sent_pred']
    
    USE = [True, False, True, False, True, False, True, True, True]
    
    newDF['Preds_Agree3'] = np.where((newDF['CNN_TE_pred']==newDF['CNN_Glove_pred'])
                                    & (newDF['CNN_Glove_pred']==newDF['CNN_w2V_pred']),1,0)
    newDF['Preds_Agree2'] = np.where((newDF['CNN_TE_pred']==newDF['CNN_Glove_pred'])
                                    | (newDF['CNN_Glove_pred']==newDF['CNN_w2V_pred'])
                                    | (newDF['CNN_TE_pred']==newDF['CNN_w2V_pred']),1,0)
    
    newDF['USE_Preds_Agree3'] = np.where((newDF['CNN_TE_USE_pred']==newDF['CNN_Glove_USE_pred'])
                                    & (newDF['CNN_Glove_USE_pred']==newDF['CNN_w2V_USE_pred']),1,0)
    newDF['USE_Preds_Agree2'] = np.where((newDF['CNN_TE_USE_pred']==newDF['CNN_Glove_USE_pred'])
                                    | (newDF['CNN_Glove_USE_pred']==newDF['CNN_w2V_USE_pred'])
                                    | (newDF['CNN_TE_USE_pred']==newDF['CNN_w2V_USE_pred']),1,0)
    
    newDF['dis_3'] = np.where((newDF['Preds_Agree3']==newDF['USE_Preds_Agree3'])
                                    & (((newDF['CNN_Glove_USE_pred']!=newDF['sentiment']) & 
                                      (newDF['CNN_Glove_pred']==newDF['sentiment'])) |
                                    ((newDF['CNN_Glove_USE_pred']==newDF['sentiment']) & 
                                      (newDF['CNN_Glove_pred']!=newDF['sentiment']))),1,0)
    return n, newDF

In [5]:
n, test_results = checkDifs(test_results)
USE_imp_test = test_results.loc[(test_results['USE_Preds_Agree3'] == 1) & (test_results['Preds_Agree3'] == 1)]

In [6]:
USE_imp_test = USE_imp_test.drop(['Average_Sent_pred','Individual_Sent_pred','Preds_Agree2','USE_Preds_Agree2',], axis = 1)

In [7]:
count_use = USE_imp_test.loc[((USE_imp_test['dis_3'] == 1) & (USE_imp_test['CNN_TE_USE_pred'] == USE_imp_test['sentiment']))]
count_norm = USE_imp_test.loc[((USE_imp_test['dis_3'] == 1) & (USE_imp_test['CNN_TE_pred'] == USE_imp_test['sentiment']))]


In [8]:
n_use = count_use.shape[0]
n_norm = count_norm.shape[0]


In [9]:
n_use

148

In [10]:
n_norm

48

## Longest Sentence Where USE + embeddings correct

Looking at the data, below. The models that included the universal sentence encoder seems to have captured sentiment better (unanimously) than the model without the sentence encoder. This could be because of a number of reasons. For one, the inclusion of stop words likely affects how the CNN interprets this review, especially because the review clearly changes sentiment throughout. In either case, it is interesting that the inclusion of the embeddings from the USE allowed the model to tease out that this was a positive sentiment when the regular CNNs (with only word level embeddings) were unable to get this one correct.

In [11]:
check_lens = [len(words) for words in count_use['sentenceWords']]

med_use = np.median(check_lens)
print(med_use)
print(list(count_use['sentenceWords'][:200].iloc[[np.argmax(check_lens)]]))
print('\n_________________\n')
print(list(count_use['reviewText'].iloc[[np.argmax(check_lens)]]))

75.0
[['five', 'stars', 'me', 'for', 'a', 'book', 'written', 'by', 'and', 'for', 'men', 'me', 'the', 'feminist', 'yep', 'yep', 'and', 'yep', 'and', 'there', 'was', 'need', 'for', 'better', 'editing', 'still', 'five', 'whole', 'stars', 'this', 'book', 'addresses', 'what', 'i', 'fight', 'for', 'equality', 'i', 'have', 'spent', 'my', 'last', 'few', 'month', 'making', 'gifts', 'either', 'crocheting', 'or', 'knitting', 'lots', 'of', 'websites', 'and', 'ebooks', 'came', 'and', 'blurred', 'before', 'my', 'eyes', 'most', 'of', 'the', 'books', 'assume', 'you', 'are', 'a', 'woman', 'a', 'house', 'wifely', 'type', 'with', 'no', 'real', 'men', 'in', 'your', 'life', 'to', 'make', 'things', 'for', 'they', 'are', 'all', 'dedicated', 'to', 'girly', 'girls', 'with', 'squishy', 'babies', 'hey', 'been', 'there', 'done', 'that', 'with', 'four', 'full', 'grown', 'off', 'spring', 'three', 'males', 'one', 'female', 'to', 'prove', 'it', 'this', 'book', 'will', 'not', 'be', 'wiped', 'off', 'my', 'kindle', 'or'

## Confusion Matrix


In [36]:
def createConfusion(data, model):
    
    y_actu = pd.Series(data['sentiment'], name='Actual Sentiment')
    y_pred = pd.Series(data[model], name='Predicted')
    
    conf = pd.crosstab(y_actu, y_pred)
    
    true_pos = np.diag(conf) 
    print(model)
    print('Accuracy: {}'.format(np.sum(true_pos)/data.shape[0]))
    
    precision = np.sum(true_pos[1] / np.sum(conf[1], axis=0))
    recall = np.sum(true_pos[1] / np.sum(conf.iloc[1]))
    
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))

    return conf


df_confusion_CNN_G = createConfusion(test_results,'CNN_Glove_USE_pred')
df_confusion_CNN_G

CNN_Glove_USE_pred
Accuracy: 0.9481791666666667
Precision: 0.9532747300907705
Recall: 0.9425583333333334


Predicted,0,1
Actual Sentiment,,
0,114456,5544
1,6893,113107


In [37]:
df_confusion_CNN_G = createConfusion(test_results,'CNN_Glove_pred')
df_confusion_CNN_G

CNN_Glove_pred
Accuracy: 0.9338166666666666
Precision: 0.9616380533484676
Recall: 0.9036833333333333


Predicted,0,1
Actual Sentiment,,
0,115674,4326
1,11558,108442


In [40]:
df_confusion_CNN_G = createConfusion(imdb_results,'CNN_Glove_USE_pred')
df_confusion_CNN_G

CNN_Glove_USE_pred
Accuracy: 0.86076
Precision: 0.8890518505737209
Recall: 0.8244


Predicted,0,1
Actual Sentiment,,
0,11214,1286
1,2195,10305


In [42]:
df_confusion_CNN_G = createConfusion(imdb_results,'CNN_Glove_pred')
df_confusion_CNN_G

CNN_Glove_pred
Accuracy: 0.82912
Precision: 0.9092717867091126
Recall: 0.7312


Predicted,0,1
Actual Sentiment,,
0,11588,912
1,3360,9140
